# Exploring ICS ATT&CK

## Query ATT&CK

### Import TAXII Libraries
ATT&CK users can use the initial Server class to instantiate a server object pointing to the framework’s public TAXII server URL https://cti-taxii.mitre.org/taxii/

In [1]:
from taxii2client.v21 import Server

In [9]:
server = Server("https://attack-taxii.mitre.org/taxii2/")

Available API Roots can be referenced from the server object. API Roots are logical groupings of TAXII Channels and Collections and can be thought of as instances of the TAXII API available at different URLs, where each API Root is the “root” URL of that particular instance of the TAXII API:

In [10]:
server.api_roots

In [11]:
api_root = server.api_roots[0]

### Explore ATT&CK TAXII Collections
The **collections** attribute can then be used and get more information about them via their respective available properties:

In [12]:
api_root.collections

In [14]:
for collection in api_root.collections:
    print(collection.title, "->", collection.description)

Enterprise ATT&CK -> ATT&CK for Enterprise provides a knowledge base of real-world adversary behavior targeting traditional enterprise networks. ATT&CK for Enterprise covers the following platforms: Windows, macOS, Linux, PRE, Office 365, Google Workspace, IaaS, Network, and Containers.
ICS ATT&CK -> The ATT&CK for Industrial Control Systems (ICS) knowledge base categorizes the unique set of tactics, techniques, and procedures (TTPs) used by threat actors in the ICS technology domain. ATT&CK for ICS outlines the portions of an ICS attack that are out of scope of Enterprise and reflects the various phases of an adversary’s attack life cycle and the assets and systems they are known to target.
Mobile ATT&CK -> ATT&CK for Mobile is a matrix of adversary behavior against mobile devices (smartphones and tablets running the Android or iOS/iPadOS operating systems). ATT&CK for Mobile builds upon NIST's Mobile Threat Catalogue and also contains a separate matrix of network-based effects, which

In [17]:
api_root.collections[1].title

'ICS ATT&CK'

In [19]:
api_root.collections[1].id

'x-mitre-collection--90c00720-636b-4485-b342-8751d232bf09'

## Set ICS ATT&CK TAXII Collection ID Variable

In [23]:
ICS_ATTACK = "x-mitre-collection--90c00720-636b-4485-b342-8751d232bf09"

## Initialize TAXII Collection Sources
According to [STIX2 docs](https://stix2.readthedocs.io/en/latest/index.html), the [TAXIICollectionSource API](https://stix2.readthedocs.io/en/latest/api/datastore/stix2.datastore.taxii.html#stix2.datastore.taxii.TAXIICollectionSource) provides an interface for searching/retrieving STIX objects from a local/remote TAXII Collection endpoint. In our case, we are pointing to our ATT&CK TAXII Collection instances (https://cti-taxii.mitre.org/stix/collections/<Collection ID>)

In [24]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client.v21 import Collection

In [25]:
ATTACK_STIX_COLLECTIONS = "https://attack-taxii.mitre.org/api/v21/collections/"
ICS_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + ICS_ATTACK + "/")
TC_ICS_SOURCE = TAXIICollectionSource(ICS_COLLECTION)

## Retrieve all ICS Techniques
Now that we can query the ICS ATT&CK TAXIICollection. We can use the query method and a set of filter to retrieve STIX objects of type "attack-pattern" -> "Techniques"

In [27]:
ICS_TECHNIQUES = TC_ICS_SOURCE.query(Filter("type", "=", "attack-pattern"))
ICS_TECHNIQUES[0]

AttackPattern(type='attack-pattern', spec_version='2.1', id='attack-pattern--008b8f56-6107-48be-aa9f-746f927dbb61', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-05-21T17:43:26.506Z', modified='2023-10-13T17:56:58.380Z', name='Block Command Message', description='Adversaries may block a command message from reaching its intended target to prevent command execution. In OT networks, command messages are sent to provide instructions to control system devices. A blocked command message can inhibit response functions from correcting a disruption or unsafe condition. (Citation: Bonnie Zhu, Anthony Joseph, Shankar Sastry 2011)  (Citation: Electricity Information Sharing and Analysis Center; SANS Industrial Control Systems March 2016)', kill_chain_phases=[KillChainPhase(kill_chain_name='mitre-ics-attack', phase_name='inhibit-response-function')], revoked=False, external_references=[ExternalReference(source_name='mitre-attack', url='https://attack.mitre.org/tech

In [28]:
for TECHNIQUE in ICS_TECHNIQUES:
    print(TECHNIQUE['external_references'][0]['external_id'], "--", TECHNIQUE['name'])

T0803 -- Block Command Message
T0823 -- Graphical User Interface
T0846 -- Remote System Discovery
T0890 -- Exploitation for Privilege Escalation
T0859 -- Valid Accounts
T0891 -- Hardcoded Credentials
T0852 -- Screen Capture
T0847 -- Replication Through Removable Media
T0843 -- Program Download
T0849 -- Masquerading
T0857 -- System Firmware
T0882 -- Theft of Operational Information
T0826 -- Loss of Availability
T0834 -- Native API
T0848 -- Rogue Master
T0818 -- Engineering Workstation Compromise
T0844 -- Program Organization Units
T0808 -- Control Device Identification
T0874 -- Hooking
T0875 -- Change Program State
T0827 -- Loss of Control
T0820 -- Exploitation for Evasion
T0830 -- Adversary-in-the-Middle
T0870 -- Detect Program State
T0806 -- Brute Force I/O
T0822 -- External Remote Services
T0812 -- Default Credentials
T0866 -- Exploitation of Remote Services
T0883 -- Internet Accessible Device
T0889 -- Modify Program
T0892 -- Change Credential
T0893 -- Data from Local System
T0856 --

## ICS ATT&CK Available since attackcti 0.3.4.3
Reference: https://pypi.org/project/attackcti/

In [29]:
from attackcti import attack_client
lift = attack_client()

ICS_TECHNIQUES = lift.get_ics_techniques()
print("Techniques Count:",len(ICS_TECHNIQUES))

Techniques Count: 83


In [30]:
ICS_TECHNIQUES[0]

AttackPattern(type='attack-pattern', spec_version='2.1', id='attack-pattern--008b8f56-6107-48be-aa9f-746f927dbb61', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-05-21T17:43:26.506Z', modified='2023-10-13T17:56:58.380Z', name='Block Command Message', description='Adversaries may block a command message from reaching its intended target to prevent command execution. In OT networks, command messages are sent to provide instructions to control system devices. A blocked command message can inhibit response functions from correcting a disruption or unsafe condition. (Citation: Bonnie Zhu, Anthony Joseph, Shankar Sastry 2011)  (Citation: Electricity Information Sharing and Analysis Center; SANS Industrial Control Systems March 2016)', kill_chain_phases=[KillChainPhase(kill_chain_name='mitre-ics-attack', phase_name='inhibit-response-function')], revoked=False, external_references=[ExternalReference(source_name='mitre-attack', url='https://attack.mitre.org/tech

## Get All Data Sources Mapped to ICS ATT&CK Techniques

In [31]:
ICS_DATA_SOURCES = []
for TECHNIQUE in ICS_TECHNIQUES:
    if 'x_mitre_data_sources' in TECHNIQUE.keys():
        for DS in TECHNIQUE['x_mitre_data_sources']:
            if DS not in ICS_DATA_SOURCES:
                ICS_DATA_SOURCES.append(DS)
ICS_DATA_SOURCES

['Process: Process Termination',
 'Operational Databases: Process History/Live Data',
 'Application Log: Application Log Content',
 'Network Traffic: Network Traffic Flow',
 'Operational Databases: Process/Event Alarm',
 'Process: Process Creation',
 'Command: Command Execution',
 'Module: Module Load',
 'Logon Session: Logon Session Creation',
 'File: File Access',
 'Network Traffic: Network Traffic Content',
 'User Account: User Account Authentication',
 'Logon Session: Logon Session Metadata',
 'Process: OS API Execution',
 'File: File Creation',
 'Drive: Drive Creation',
 'Operational Databases: Device Alarm',
 'Service: Service Creation',
 'File: File Modification',
 'Scheduled Job: Scheduled Job Modification',
 'File: File Metadata',
 'Firmware: Firmware Modification',
 'Windows Registry: Windows Registry Key Modification',
 'Script: Script Execution',
 'Network Share: Network Share Access',
 'Network Traffic: Network Connection Creation',
 'Service: Service Metadata',
 'Windows 

## Get All Groups from ICS ATT&CK

In [32]:
ICS_GROUPS = lift.get_ics_groups()
for GROUP in ICS_GROUPS:
    print(GROUP['name'])

HEXANE
APT33
Wizard Spider
Lazarus Group
GOLD SOUTHFIELD
CyberAv3ngers
TEMP.Veles
OilRig
Sandworm Team
FIN7
FIN6
Dragonfly
ALLANITE
APT38
Leafminer


## Get All Malware from ICS ATT&CK

In [33]:
ICS_MALWARE = lift.get_ics_malware()
for MALWARE in ICS_MALWARE:
    print(MALWARE['name'])

Conficker
NotPetya
BlackEnergy
PLC-Blaster
Conficker
Stuxnet
Bad Rabbit
Bad Rabbit
Industroyer
Stuxnet
Backdoor.Oldrea
EKANS
LockerGoga
INCONTROLLER
Flame
Industroyer
KillDisk
REvil
ACAD/Medre.A
Ryuk
EKANS
Fuxnet
BlackEnergy 3
Triton
WannaCry
Killdisk
Industroyer2
Duqu
VPNFilter
